In [1]:
%cd ..

/home/akajal/DiVAE


In [2]:
# Python imports
import os
import sys

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# ML imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# DiVAE imports
from models.rbm.chimeraRBM import ChimeraRBM
from models.rbm.qimeraRBM import QimeraRBM
from models.rbm.rbm import RBM
from models.samplers.pcd import PCD
from models.autoencoders.gumboltCaloCRBM import GumBoltCaloCRBM

from nbutils import *

# DWave imports
from dwave.system import DWaveSampler, LeapHybridSampler
import neal

[15:20:19.591] INFO   DiVAE                                             Willkommen!
[15:20:19.592] INFO   DiVAE                                             Loading configuration.
2022-03-11 15:20:20,833 dwave.cloud INFO  MainThread Log level for 'dwave.cloud' namespace set to 0
[15:20:20.833] INFO   dwave.cloud                                       Log level for 'dwave.cloud' namespace set to 0


In [3]:
def load_state(model, run_path, device):
        model_loc = run_path
        
        # Open a file in read-binary mode
        with open(model_loc, 'rb') as f:
            # Interpret the file using torch.load()
            checkpoint=torch.load(f, map_location=device)
            
            local_module_keys=list(model._modules.keys())
            for module in checkpoint.keys():
                if module in local_module_keys:
                    print("Loading weights for module = ", module)
                    getattr(model, module).load_state_dict(checkpoint[module])

In [4]:
def init_model(cfg):
    model = GumBoltCaloCRBM(flat_input_size=[504],
                            train_ds_mean=0.,
                            activation_fct=torch.nn.ReLU(),
                            cfg=cfg)
    return model

In [5]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="config")

/opt/conda/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [6]:
model = init_model(cfg)
model.create_networks()

2022-03-11 15:20:21,612 dwave.cloud.client.base INFO  MainThread Fetching definitions of all available solvers
[15:20:21.612] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
2022-03-11 15:20:22,598 dwave.cloud.client.base INFO  MainThread Adding solver StructuredSolver(id='DW_2000Q_6')
[15:20:22.598] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(id='DW_2000Q_6')
2022-03-11 15:20:22,811 dwave.cloud.client.base INFO  MainThread Adding solver StructuredSolver(id='Advantage_system4.1')
[15:20:22.811] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(id='Advantage_system4.1')
2022-03-11 15:20:22,915 dwave.cloud.client.base INFO  MainThread Adding solver StructuredSolver(id='Advantage_system5.1')
[15:20:22.915] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(id='Advantage_system5.1')


Initializing NetworkV3


In [7]:
run_path = '/home/akajal/DiVAE/outputs/2022-01-24/14-14-25/wandb/run-20220124_141425-3o4vumug/files/GumBoltCaloCRBM_calo_default_latest.pth'
load_state(model, run_path, 'cuda:{0}'.format(cfg.gpu_list[0]))

Loading weights for module =  _activation_fct
Loading weights for module =  _bce_loss
Loading weights for module =  _energy_activation_fct
Loading weights for module =  _hit_activation_fct
Loading weights for module =  _output_loss
Loading weights for module =  _hit_loss
Loading weights for module =  _hit_smoothing_dist_mod
Loading weights for module =  encoder
Loading weights for module =  prior
Loading weights for module =  decoder
Loading weights for module =  sampler


In [8]:
def beta_estimation(num_iterations=10, lr=0.1, beta_init=5., n_reads=100, crbm_sampler=None, dwave_sampler=None):
    assert crbm_sampler is not None
    assert dwave_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    crbm = crbm_sampler.get_rbm()
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()
    crbm_energy_exp = torch.mean(sample_energies(crbm, crbm_vis, crbm_hid))
    
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    n_vis = len(crbm.visible_qubit_idxs)
    qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}
    
    for i in range(num_iterations):
        dwave_weights, dwave_vbias, dwave_hbias = -ising_weights/beta, -ising_vbias/beta, -ising_hbias/beta
        dwave_weights_np = dwave_weights.detach().cpu().numpy()
        biases = torch.cat((dwave_vbias, dwave_hbias)).detach().cpu().numpy()
        
        print("Iteration = {0}, beta = {1} : J range = ({2}, {3}), h range = ({4}, {5})".format(i, beta, np.min(dwave_weights_np),
                                                                                                np.max(dwave_weights_np), np.min(biases),
                                                                                                np.max(biases)))
        
        h = {qubit_idx:bias for qubit_idx, bias in zip(qubit_idxs, biases)}
        J = {}
        for edge in crbm_edgelist:
            if edge[0] in crbm.visible_qubit_idxs:
                J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
            else:
                J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                    
        response = dwave_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
        dwave_samples, dwave_energies = batch_dwave_samples(response)
        
        # Transform QPU Ising samples into RBM binary samples
        dwave_vis, dwave_hid = dwave_samples[:, :n_vis], dwave_samples[:, n_vis:]
        dwave_vis = torch.from_numpy(np.where(dwave_vis == -1., 0., dwave_vis)).float()
        dwave_hid = torch.from_numpy(np.where(dwave_hid == -1., 0., dwave_hid)).float()
        
        dwave_energy_exp = torch.mean(sample_energies(crbm, dwave_vis, dwave_hid))
        
        print("crbm_energy_exp : {0}, dwave_energy_exp : {1}".format(crbm_energy_exp, dwave_energy_exp))
        beta = beta + lr*(float(crbm_energy_exp)-float(dwave_energy_exp))
        betas.append(beta)
        
    return betas

In [9]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})
betas = beta_estimation(num_iterations=10, beta_init=10., lr=0.001, crbm_sampler=model.sampler, dwave_sampler=qpu_sampler)

2022-03-11 15:20:27,517 dwave.cloud.client.base INFO  MainThread Fetching definitions of all available solvers
[15:20:27.517] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
2022-03-11 15:20:28,628 dwave.cloud.client.base INFO  MainThread Adding solver StructuredSolver(id='DW_2000Q_6')
[15:20:28.628] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(id='DW_2000Q_6')
2022-03-11 15:20:28,730 dwave.cloud.client.base INFO  MainThread Adding solver StructuredSolver(id='Advantage_system4.1')
[15:20:28.730] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(id='Advantage_system4.1')
2022-03-11 15:20:28,835 dwave.cloud.client.base INFO  MainThread Adding solver StructuredSolver(id='Advantage_system5.1')
[15:20:28.835] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(id='Advantage_system5.1')


Iteration = 0, beta = 10.0 : J range = (-0.12976184487342834, 0.09670235961675644), h range = (-0.33781254291534424, 0.4295719563961029)
crbm_energy_exp : -4012.7861328125, dwave_energy_exp : -1149.5977783203125
Iteration = 1, beta = 7.136811645507812 : J range = (-0.18182048201560974, 0.13549798727035522), h range = (-0.47333818674087524, 0.6019101738929749)
crbm_energy_exp : -4012.7861328125, dwave_energy_exp : -1168.358154296875
Iteration = 2, beta = 4.292383666992187 : J range = (-0.30230721831321716, 0.2252882570028305), h range = (-0.7870045304298401, 1.0007771253585815)
crbm_energy_exp : -4012.7861328125, dwave_energy_exp : -1161.0228271484375
Iteration = 3, beta = 1.4406203613281243 : J range = (-0.9007359743118286, 0.6712549924850464), h range = (-2.3449103832244873, 2.981854200363159)


SolverFailureError: Linear coefficients out of h_range.

In [ ]:
plot_betas(betas)

In [ ]:
def energy_deltas_rbm(beta_min=1, beta_max=100, n_reads=100, crbm_sampler=None, dwave_sampler=None):
    assert crbm_sampler is not None
    assert dwave_sampler is not None
    
    betas = []
    deltas = []
    
    crbm = crbm_sampler.get_rbm()
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()
    crbm_energy_exp = torch.mean(sample_energies(crbm, crbm_vis, crbm_hid))
    
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    n_vis = len(crbm.visible_qubit_idxs)
    qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}
    
    for beta in range(beta_min, beta_max):
        beta = float(beta)
        dwave_weights, dwave_vbias, dwave_hbias = -ising_weights/beta, -ising_vbias/beta, -ising_hbias/beta
        dwave_weights_np = dwave_weights.detach().cpu().numpy()
        biases = torch.cat((dwave_vbias, dwave_hbias)).detach().cpu().numpy()
        
        print("beta = {0} : J range = ({1}, {2}), h range = ({3}, {4})".format(beta, np.min(dwave_weights_np),
                                                                               np.max(dwave_weights_np), np.min(biases),
                                                                               np.max(biases)))
        
        h = {qubit_idx:bias for qubit_idx, bias in zip(qubit_idxs, biases)}
        J = {}
        for edge in crbm_edgelist:
            if edge[0] in crbm.visible_qubit_idxs:
                J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
            else:
                J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                    
        response = dwave_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
        dwave_samples, dwave_energies = batch_dwave_samples(response)
        
        # Transform QPU Ising samples into RBM binary samples
        dwave_vis, dwave_hid = dwave_samples[:, :n_vis], dwave_samples[:, n_vis:]
        dwave_vis = torch.from_numpy(np.where(dwave_vis == -1., 0., dwave_vis)).float()
        dwave_hid = torch.from_numpy(np.where(dwave_hid == -1., 0., dwave_hid)).float()
        
        dwave_energy_exp = torch.mean(sample_energies(crbm, dwave_vis, dwave_hid))
        
        print("crbm_energy_exp : {0}, dwave_energy_exp : {1}".format(crbm_energy_exp, dwave_energy_exp))
        betas.append(beta)
        deltas.append(float(crbm_energy_exp)-float(dwave_energy_exp))
        
    return betas, deltas

In [ ]:
betas, deltas = energy_deltas_rbm(beta_min=200, beta_max=210, crbm_sampler=model.sampler, dwave_sampler=qpu_sampler)

In [ ]:
def plot_beta_vs_delta(betas, deltas):
    """
    Plot the estimates of beta during the beta estimation procedure   
    """
    fig, ax = plt.subplots(figsize=(40, 16))
    
    plt.plot(betas, deltas)
    
    ax.set_xlabel("Beta", fontsize=60)
    ax.set_ylabel(r"$E_{p_{RBM}}[E_{b, W}] - E_{p_{HW}}[E_{b, W}]$", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.grid("True", axis='both', which='both')
    
    plt.show()
    plt.close()

In [ ]:
plot_beta_vs_delta(betas, deltas)